In [3]:
import pandas as pd
import numpy as np

In [2]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [1]:
cd C:\Users\Administrator\Desktop\ML assignment\Assignment3\ml-latest-small

C:\Users\Administrator\Desktop\ML assignment\Assignment3\ml-latest-small


In [5]:
links = pd.read_csv('links.csv')
movie = pd.read_csv('movies.csv')
rating = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [5]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [9]:
rating_pivot = rating.pivot(index='userId', columns='movieId', values='rating').fillna(0)
rating_pivot.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1. Create recommender system using ratings.csv file. 

In [10]:
reader = Reader(rating_scale=(0.5,5))

data_rating = Dataset.load_from_df(rating[['userId','movieId','rating']],reader)

In [11]:
[trainset, testset] = train_test_split(data_rating, test_size=0.15, shuffle=True)

recom = KNNWithMeans(k=50, sim_options={'name':'cosine','user_based':True})

recom.fit(data_rating.build_full_trainset())

test_pred = recom.test(testset)

RMSE = accuracy.rmse(test_pred)
print('RMSE = ', RMSE)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.7799
RMSE =  0.7799435129166931


### 2. Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [12]:
uid = int(input('Enter the UserId : '))
mid = int(input('Enter the MovieId : '))
pre = recom.predict(uid, mid)
print('The rating value for MovieId ',mid,' by UserId ',uid, ' is ',pre[3])

Enter the UserId : 6
Enter the MovieId : 339
The rating value for MovieId  339  by UserId  6  is  3.5891671404458783


### 3.	Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions). Take userID as a keyboard input. Fix the threshold rating as 2.5

In [13]:
# Keep a threshold of 2.5 and then display recommendations

def recommend(pre):
    if pre[3] >= 2.5 :
        return True
    else:
        return False

In [14]:
uid = int(input('Enter the UserId : '))
d = dict()
for i in rating_pivot.columns:
    pre = recom.predict(uid,i)
    if recommend(pre) == True :
        d[i] = pre[3]
        
final_dict = sorted(d.items(), key=lambda x: x[1], reverse=True)

Enter the UserId : 27


In [16]:
print('Top 10 movies recommend for UserId ',uid,' :')
for i in range(10):
    print(i+1,' --> ',(movie[movie['movieId'] == final_dict[i][0]]['title'].values)[0])

Top 10 movies recommend for UserId  27  :
1  -->  In the Realm of the Senses (Ai no corrida) (1976)
2  -->  Denise Calls Up (1995)
3  -->  Hype! (1996)
4  -->  Vampire in Venice (Nosferatu a Venezia) (Nosferatu in Venice) (1986)
5  -->  Four Days in September (O Que É Isso, Companheiro?) (1997)
6  -->  Ballad of Narayama, The (Narayama bushiko) (1983)
7  -->  Red Sorghum (Hong gao liang) (1987)
8  -->  Babes in Toyland (1934)
9  -->  Black Tar Heroin: The Dark End of the Street (2000)
10  -->  On the Beach (1959)


### 4.	Display the MovieID, IMDB ID, Average User Rating (excluding predictions), genres and tag of all the movies found in Step 3 as a data frame.

In [17]:
avg = rating.groupby('movieId')['rating'].mean().reset_index()
avg = avg.rename(columns = {'rating':'Avg_Rating'})
avg.head()

,movieId,Avg_Rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429


In [18]:
df_dict = {}
recommend_movie = pd.DataFrame()

for i in range(10):
    val = final_dict[i][0]
    moviename = (movie[movie['movieId'] == val]['title'].values)[0]
    imdb = (links[links['movieId'] == val]['imdbId'].values)[0]
    avg_user_rating = (avg[avg['movieId'] == val]['Avg_Rating'].values)[0]
    genres = (movie[movie['movieId'] == val]['genres'].values)[0]
    
    tag = tags[tags['movieId'] == val]['tag'].values
    if len(tag) == 0:
        tag = 'NaN'
    else:
        tag = (tag)[0]

    df_dict = {'Movie_Id':val, 'IMDB_ID':imdb,
               'Avg_User_Rating':avg_user_rating,
              'Genres':genres, 'Tag':tag, 'Movie_Name':moviename}
    recommend_movie = recommend_movie.append(df_dict,ignore_index=True)

ls = ['Movie_Id','Movie_Name','IMDB_ID','Genres','Tag','Avg_User_Rating']
recommend_movie[ls]

,Movie_Id,Movie_Name,IMDB_ID,Genres,Tag,Avg_User_Rating
0,495.0,In the Realm of the Senses (Ai no corrida) (1976),74102.0,Drama,NaN,5.0
1,633.0,Denise Calls Up (1995),112844.0,Comedy,NaN,5.0
2,1310.0,Hype! (1996),116589.0,Documentary,NaN,5.0
3,1349.0,Vampire in Venice (Nosferatu a Venezia) (Nosfe...,91651.0,Horror,NaN,5.0
4,1759.0,"Four Days in September (O Que É Isso, Companhe...",119815.0,Drama,NaN,5.0
5,2512.0,"Ballad of Narayama, The (Narayama bushiko) (1983)",84390.0,Drama,NaN,5.0
6,2972.0,Red Sorghum (Hong gao liang) (1987),93206.0,Drama|War,NaN,5.0
7,3086.0,Babes in Toyland (1934),24852.0,Children|Comedy|Fantasy|Musical,NaN,5.0
8,3303.0,Black Tar Heroin: The Dark End of the Street (...,221023.0,Documentary,NaN,5.0
9,3379.0,On the Beach (1959),53137.0,Drama,NaN,4.5


----------------------------------------------------------------------------------------END---------------------------------------------------------------------------------------------